In [1]:
import pymc3 as pm 
import matplotlib.pyplot as plt
import numpy as np 
import theano
import theano.tensor as tt
from scipy import stats
import pymc3 as pm 
from ipynb.fs.full.simul_data import *
from ipynb.fs.full.counterfactual_generate import *
a_dim=7
mu_a = np.zeros(a_dim)
cov_a = np.eye(a_dim)

trans_dim = 6
u_dim = 3
rating_dim = 5
N=100
mu_u = np.zeros(u_dim)
cov_u = np.eye(u_dim)
mu_trans = np.zeros(trans_dim)
cov_trans = np.eye(trans_dim)
mu_rating = np.zeros(rating_dim)
cov_rating = np.eye(rating_dim)

In [2]:
num_samples =100
causal_model = model1()
data = causal_model.generate(num_samples)
print(data["transcript"].shape)
# mu_rand = np.random.normal(size=10)

(100,)


In [3]:
def model_fit(data,method='mcmc'):

    with pm.Model() as model:

        #prior sampling
        u = pm.MvNormal('u',mu=mu_u,cov=cov_u,shape=u_dim)
        transcript0 = pm.MvNormal('transcript0',mu=mu_trans,cov=cov_trans)
        view0 = pm.Normal('view0',mu=0,tau=1)

        #effect of u 
        eta_u_transcript = pm.MatrixNormal('eta_u_transcript',colcov=cov_u,
                       rowcov=cov_trans, shape=(trans_dim, u_dim))
        eta_u_view = pm.MvNormal('eta_u_view',mu=mu_u,cov=cov_u,shape=u_dim)
        eta_u_rating = pm.MatrixNormal('eta_u_rating',colcov=cov_u,
                       rowcov=cov_rating, shape=(rating_dim, u_dim))

        #effect of protected attribute
        eta_a_transcript = pm.MatrixNormal('eta_a_transcript',colcov=cov_a,
                       rowcov=cov_trans, shape=(a_dim, trans_dim))
        eta_a_view = pm.MvNormal('eta_a_view',mu=mu_a,cov=cov_a,shape=a_dim)
        eta_a_rating = pm.MvNormal('eta_a_rating',colcov=cov_a,
                       rowcov=cov_rating, shape=(rating_dim, a_dim))

        #effect of transcript on view and rating
        eta_transcript_view = pm.MvNormal('eta_transcript_view',mu=mu_trans,cov=cov_trans)
        eta_transcript_rating = pm.MatrixNormal('eta_transcript_rating',colcov=cov_trans,
                       rowcov=cov_rating, shape=(rating_dim, trans_dim))

        #effect of view on rating
        eta_view_rating = pm.MvNormal('eta_view_rating',mu=mu_rating,co=cov_rating,shape=rating_dim)


        sigma_transcript_sq = pm.InverseGamma('sigma_transcript_sq',alpha=1,beta=1)
        sigma_rating_sq = pm.InverseGamma('sigma_rating_sq',alpha=1,beta=1)

        transcript_mean = transcript0 + tt.dot(eta_u_transcript ,u) + tt.dot(data['a'] , eta_a_transcript)
        #print(transcript_mean.shape)
        transcript = pm.MvNormal('transcript', mu= transcript_mean, cov = sigma_transcript_sq*np.eye(trans_dim), observed = data["transcript"] )

        view_mean = tt.exp(view0 + eta_u_view * u +  transcript * eta_transcript_view + tt.dot(data['a'],eta_a_view))
        view = pm.Poisson('view',mu =view_mean, observed = data['view'] )

        rating_mean = eta_u_rating * u +  tt.dot(data['a'],eta_a_rating) + transcript* eta_transcript_rating + view* eta_view_rating 
        rating = pm.Normal('rating', mu= rating_mean, sd = sigma_rating_sq, observed = data["rating"] ) 
        
        if method == 'mcmc':        
            trace=pm.sample(1000)
        elif method == 'vi':
            trace=pm.fit()
        else:
            print('Go Fuck Yourself')
            
    return trace
    

In [4]:
trace = model_fit(data)

AttributeError: 'int' object has no attribute 'shape'

In [ ]:
trace["eta_view_rating"].mean(axis=0)

In [ ]:
u_list = counterfactual_sample(data,trace)

In [ ]:
pm.traceplot(trace)

In [ ]:
u_list.shape